# Import Library

In [1]:
!pip install -q stockstats

In [2]:
!pip install -q catboost

     |████████████████████████████████| 76.6 MB 45 kB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

volatility_folder = "/content/drive/MyDrive/fyp/Final report/4. Backtesting/Volatility_result/"
backtest_summary_folder = volatility_folder + "backtest_summary/"
record_folder = volatility_folder + "trade_record/"
user_test_folder = volatility_folder + "user_test/"
model_folder = "/content/drive/MyDrive/fyp/Final report/4. Backtesting/Catboost/"

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import floor
import pickle
from catboost import CatBoostRegressor

import torch.nn as nn
import os.path as osp
import torch.utils.data as data
import torch.nn.functional as F

from tqdm import tqdm
import time, matplotlib, torch, os
from statsmodels.tsa.arima_model import ARIMA
from torch.utils.tensorboard import SummaryWriter

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
import os
from os import walk

import plotly
%matplotlib inline
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime


In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [7]:
#For DNN / CNN+LSTM
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Flatten, Dense

# Backtesting Setting

In [8]:
start_date = []
num_period = 10
period_year = 1

start = 2022-(num_period+period_year-1)
for i in range(num_period):
    start_date.append((start+i)*10000+101)


date_range_list={}
for i in start_date:
    date_range_list[i] = (i-101)+(period_year-1)*10000+1231

print(date_range_list)

stock_list = {"BlockChain":["COIN","NVDA","FB"],
              "Airline":["BA","GD","LMT",],
              "Traveling":["UBER","ABNB","MAR","BKNG"],
              "Semiconductors":["INTC","NVDA","QCOM","MU","AMD"],
              "Cloud Computing":["IBM", "AMZN","GOOG","CRM"],
              "Social Media":["TWTR","SNAP","PINS","FB"],
              "Entertainment":["DIS","NFLX","FB"],
              "Retail":["WMT","COST","TGT","BBY","HD"],
              "Franchise":["MCD", "YUM", "SBUX", "DPZ"],
              "Real Estate":["HST", "EQR", "AVB", "PLD", "SPG"],
              "Telecommunication":["T","TMUS","VZ","CMCSA","CHTR"],
              "Energy & Resources":["DOW", "DD"],
              "Luxury goods":["RACE","EL","PVH"]
                            }
#stock_list = {"Entertainment":["DIS","NFLX","FB"]}

start_date = "2000-01-01"
end_date = "2022-12-31"

do_nth_name = "Buy&Hold"
model_name = "WithModel"

backtest_col = ["Sector","Stock",
                "Range_Start","Range_End",
                do_nth_name+"_Profit",model_name+"_Profit",
                do_nth_name+"_CAGR(in %)",model_name+"_CAGR(in %)",
                "CAGR_Performance(in %)",
                do_nth_name+"_MDD(in %)",model_name+"_MDD(in %)",
                "MDD_Performance(in %)"]

{20120101: 20121231, 20130101: 20131231, 20140101: 20141231, 20150101: 20151231, 20160101: 20161231, 20170101: 20171231, 20180101: 20181231, 20190101: 20191231, 20200101: 20201231, 20210101: 20211231}


# Backtesting Module

In [9]:
################### Input ##########################
# For hist_price_data: index=["date"], columns = ["Open"]
# For pred_action: index=["date"], columns = ["Action"] (Buy/Sell)
################### Output #########################
# 1. trading record
# 2. total profit
class backtest:
    hpd = ""
    pred_action=pd.DataFrame()
    trade_record=pd.DataFrame(index=[],
                              columns=["Action","Price","Position","Cash","Pos_Bal","Cash_Bal","Cum_Profit","Total_Bal"],
                             )
    capital = 0
    cash_balance = 0
    profit = 0
    handle_fee = 0
    position = 0
    last_price = 0
    do_nth_profit = 0
    num_year = 0
    _tested = False
    
    _stock_trough = 0
    _stock_peak = 0
    _stock_all_time_low = 0
    _stock_all_time_high = 0
    
    _portfolio_trough = 0
    _portfolio_peak = 0
    _portfolio_all_time_low = 0
    _portfolio_all_time_high = 0
    
    def __init__(self,hist_price_data,pred_action,capital,handling_fee,num_year=1):
        self.hpd = hist_price_data
        self.pred_action = pred_action
        self.capital = capital
        self.cash_balance = capital
        self._portfolio_trough = capital
        self._portfolio_peak = capital
        self._portfolio_all_time_low = capital
        self._portfolio_all_time_high = capital
        self.handle_fee = handling_fee
        self.num_year = num_year  
        
    def clear_trade_record(self,sec):
        self.trade_record=pd.DataFrame(index=[],
                                       columns=["Action","Price","Position","Cash","Pos_Bal","Cash_Bal","Cum_Profit","Total_Bal"],
                                       )
        print("Clearing trade_record...")
        time.sleep(sec)
        
    def start_test(self): 
        if not self._tested:
            status = "sell"
            self._tested = True
            print("Start Backtesting...")  
            self._stock_all_time_low = self.hpd.iloc[0,0]
            self._stock_all_time_high = self.hpd.iloc[0,0]
            self._stock_trough = self.hpd.iloc[0,0]
            self._stock_peak = self.hpd.iloc[0,0]
            # For loop to iterate the data
            for ind in self.pred_action.index:
                # Update latest price
                self.last_price = self.hpd.loc[ind,"Open"]
                
                # Mark All Time Low,High , Trough and Peak for MDD of stock price (using "Buy&Hold")
                if self.last_price > self._stock_all_time_high:
                    self._stock_all_time_high = self.last_price
                if self.last_price < self._stock_all_time_low:
                    self._stock_all_time_low = self.last_price
                if self.last_price > self._stock_all_time_low and self._stock_trough != self._stock_all_time_low:
                    self._stock_trough = self._stock_all_time_low
                if self.last_price < self._stock_all_time_high and self._stock_peak != self._stock_all_time_high:
                    self._stock_peak = self._stock_all_time_high
                
                # Do the action
                if self.pred_action.loc[ind,"Action"].lower() == "buy" and status == "sell":
                    self._buy(ind,self.last_price)
                    status = "buy"
                elif self.pred_action.loc[ind,"Action"].lower() == "sell" and status == "buy":
                    self._sell(ind,self.last_price)
                    status = "sell"
                else:
                    self._hold(ind,self.last_price) # newly added
                    
                
                curr_portfolio_val = self.get_capital()+self.get_profit()
                
                # Mark All Time Low,High , Trough and Peak for MDD of our portfolio balance (using our model)
                if curr_portfolio_val > self._portfolio_all_time_high:
                    self._portfolio_all_time_high = curr_portfolio_val
                if curr_portfolio_val < self._portfolio_all_time_low:
                    self._portfolio_all_time_low = curr_portfolio_val
                if curr_portfolio_val > self._portfolio_all_time_low and self._portfolio_trough != self._portfolio_all_time_low:
                    self._portfolio_trough = self._portfolio_all_time_low
                if curr_portfolio_val < self._portfolio_all_time_high and self._portfolio_peak != self._portfolio_all_time_high:
                    self._portfolio_peak = self._portfolio_all_time_high
                            
            # =================================================
            self._run_do_nothing() # Calculate do nothing profit
        else:
            print("Backtesting has been completed...")
              
                
        
    def _mark_down_record(self,date,action,price,pos_delta,cash_delta):
        self.trade_record.loc[date] = [action,price,pos_delta,cash_delta,
                                       round(self.position,4),round(self.cash_balance,3),
                                       round(self.get_profit(),3),round(self.get_amount(),3)]
        
    def _buy(self,date,price):
        # Assume use all money to buy all
        buy_pos = floor(self.cash_balance / price)
        for i in range(buy_pos+1):
            act_buy_pos = buy_pos - i
            if act_buy_pos == 0:
                #print("You do not have enough money to buy!")
                return
            total_amt = act_buy_pos*price*(1+self.handle_fee)
            if self.cash_balance > total_amt:
                self.position += act_buy_pos
                self.cash_balance -= total_amt
                self._mark_down_record(date,
                                       "Buy",
                                       price,
                                       act_buy_pos,
                                       -total_amt)
                print("Bought at",date,"with price =", price, "\tPos:", act_buy_pos)
                return
        
    
    def _sell(self,date,price):
        # Assume sell all position
        sell_pos = self.position
        total_amt = sell_pos*price*(1-self.handle_fee)
        if self.position >= 1:
            self.position -= sell_pos
            self.cash_balance += total_amt
            self._mark_down_record(date,
                                   "Sell",
                                   price,
                                   -sell_pos,
                                   total_amt)
            print("Sold at",date,"with price =", price, "\tPos:", sell_pos)
            
            return
        
    def _hold(self,date,price): # newly added
        self._mark_down_record(date,"Hold",price,0,0)

    def _run_do_nothing(self):
        balance = self.capital
        first_day_price = self.hpd.iloc[0,0]
        last_day_price = self.hpd.iloc[-1,0]
        buy_pos = floor(self.capital / first_day_price) # Calculate how many position can buy
        total_amt = buy_pos*first_day_price*(1+self.handle_fee) # Check if okay to buy (including fee)
        while self.capital < total_amt: # If not enough, reduce buy_pos by 1
            buy_pos -= 1
            total_amt = buy_pos*first_day_price*(1+self.handle_fee)
        position = buy_pos # Buy in 
        balance -= total_amt
        self.do_nth_profit = last_day_price*position+balance-self.capital

    def get_performance(self):
        if self._tested:
            model_profit = self.get_profit()
            do_nth_profit = self.get_do_nothing()
            perf = (model_profit-do_nth_profit)/abs(do_nth_profit)
            return perf
        else:
            print("No Backtesting Record.")

    def get_do_nothing(self):
        if self._tested:
            return self.do_nth_profit
        else:
            print("No Backtesting Record.")

    def get_profit(self):
        if self._tested:
            return self.get_cash_balance()+self.get_last_price()*self.get_position()-self.get_capital()
        else:
            print("No Backtesting Record.")

    def get_last_price(self):
        if self._tested:
            return self.last_price
        else:
            print("No Backtesting Record.")
    
    def get_position(self):
        if self._tested:
            return self.position
        else:
            print("No Backtesting Record.")
    
    def get_do_nothing_CAGR(self):
        if self._tested:
            start_bal = self.get_capital()
            end_bal = self.get_do_nothing()+self.get_capital()
            num_year = self.get_num_year()
            return pow(end_bal/start_bal,1/num_year)-1
        else:
            print("No Backtesting Record.")
            
    def get_model_CAGR(self):
        if self._tested:
            start_bal = self.get_capital()
            end_bal = self.get_profit()+self.get_capital()
            num_year = self.get_num_year()
            return pow(end_bal/start_bal,1/num_year)-1
        else:
            print("No Backtesting Record.")
            
    def get_diff_in_CAGR(self):
        if self._tested:
            return self.get_model_CAGR() - self.get_do_nothing_CAGR()
        else:
            print("No Backtesting Record.")
    
    def get_do_nothing_MDD(self):
        if self._tested:
            return (self._stock_trough-self._stock_peak)/self._stock_peak
        else:
            print("No Backtesting Record.")
            
    def get_model_MDD(self):
        if self._tested:
            return (self._portfolio_trough-self._portfolio_peak)/self._portfolio_peak
        else:
            print("No Backtesting Record.")
            
    def get_diff_in_MDD(self):
        if self._tested:
            return self.get_model_MDD() - self.get_do_nothing_MDD()
        else:
            print("No Backtesting Record.")
    
    def get_num_year(self):
        return self.num_year
    
    def get_cash_balance(self):
        return self.cash_balance
    
    def get_capital(self):
        return self.capital
    
    def get_amount(self):
        return self.get_capital()+self.get_profit()

    def print_do_nothing(self):
        if self._tested:
            print("If buy at", self.hpd.index[0],"with price =",self.hpd.iloc[0,0])
            print("and do nothing")
            print("Current Profit:",self.get_do_nothing())
        else:
            print("No Backtesting Record.")
            
    def print_performance(self):
        if self._tested:
            print("Performance:", str(round(self.get_performance()*100,2))+"%")
        else:
            print("No Backtesting Record.")
            
    def print_CAGR_performance(self):
        if self._tested:
            print("CAGR Performance:", str(round(self.get_diff_in_CAGR()*100,2))+"%")
        else:
            print("No Backtesting Record.")
    
    def print_trade_record(self):
        if self._tested:
            print(self.trade_record)
        else:
            print("No Backtesting Record.")
            print(self.trade_record)
    
    def print_profit(self):
        if self._tested:
            print("Current Profit with model:",self.get_profit())
        else:
            print("No Backtesting Record.")
    
    def export_trade_record(self,stock,add_msg = ""):
        if self._tested:
            # Save the trade record to the path
            if not add_msg == "":
                add_msg = "_" + add_msg
            self.trade_record = self.trade_record.sort_index()
            self.trade_record.to_csv(record_folder+stock+add_msg+".csv")
            print("Trade record exported.")
        else:
            print("No Backtesting Record.")

In [ ]:
def get_last_n_day_mean(df,n):
    series = df["open"]
    windows = series.rolling(n)
    n_days_averages = windows.mean()
    n_days_list = n_days_averages.tolist()
    final_list = n_days_list[n:]
    new_df = pd.DataFrame(final_list,columns=["mean_"+str(n)],index=df.index[n:])
    return new_df

# Import df

In [10]:
import pandas_datareader.data as web
from stockstats import StockDataFrame

#Setting the index values as timestamp
def process(code):
  print("Enter the code of the model that you want to implement")
  #code = input()
  #final_code = '^'+code
  final_code = code
  start_date = "2012-01-01"
  end_date = "2021-12-31"
  stock = web.DataReader(final_code , 'stooq',start=start_date, end=end_date)
  stock.columns = ["open","high","low","close","volume"]
  
  # Use online package to generate additional features
  x = StockDataFrame(stock)
  stock = x[['open','high','low','close','volume',
          'boll', 'boll_ub', 'boll_lb',
          'macd', 'macdh', 'macds',
          'rsi_11', 'rsi_14', 'rsi_21']]
  stock.index = [int(str(ind)[0:4]+str(ind)[5:7]+str(ind)[8:10]) for ind in stock.index]
  stock = stock.sort_index()

  return stock

In [11]:
stock = process("IBM")

Enter the code of the model that you want to implement


In [12]:
stock

,open,high,low,close,volume,boll,boll_ub,boll_lb,macd,macdh,macds,rsi_11,rsi_14,rsi_21
20120103,128.88,130.23,128.38,128.58,8180058,128.1790,134.932612,121.425388,-1.811274,0.489102,-2.300376,51.281366,47.190995,42.920670
20120104,128.10,128.60,127.63,128.07,6297526,128.3965,135.476771,121.316229,-2.091832,0.330819,-2.422652,48.488535,44.808561,41.192557
20120105,127.56,127.71,126.38,127.46,6466200,128.6025,135.879775,121.325225,-2.367574,0.137783,-2.505356,45.064237,41.897279,39.091982
20120106,127.28,127.34,125.83,125.98,7094969,128.9115,136.502568,121.320432,-2.622973,-0.083171,-2.539802,35.625325,34.061158,33.612565
20120109,125.76,125.80,124.42,125.33,7535572,129.2665,136.943913,121.589087,-2.760113,-0.241104,-2.519009,30.883945,30.223364,31.016798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211227,129.08,130.08,128.40,130.05,4346229,131.4500,133.282321,129.617679,-0.099397,-0.058552,-0.040846,8.180361,8.545592,8.997984
20211228,130.03,131.63,130.02,131.05,3488601,131.8000,132.899939,130.700061,-0.042801,-0.026749,-0.016052,14.491914,14.911940,15.424720
20211229,130.82,132.61,130.73,131.76,4291007,132.0500,132.611070,131.488930,-0.010219,-0.007870,-0.002349,28.868360,29.305681,29.832936
20211230,132.16,132.77,131.74,132.32,3196152,132.1950,132.548553,131.841447,0.005609,0.002493,0.003116,100.000000,100.000000,100.000000


# Processing Functions


## Split the train and test data

In [13]:
# Split the train and test data
def custom_split(data,start,end):
    train = (data.index >= start) & (data.index <= end)
    train_X = data[train]
    
    return train_X

In [14]:
train_X = custom_split(stock,start = 20130101,end = 20171031)
valid_X = custom_split(stock,start = 20171101,end = 20181231)
test_X = custom_split(stock,start = 20190101,end = 20201231)

In [15]:
print(len(train_X))
print(len(valid_X))
print(len(test_X))

1218
292
505


## Label the target result (opening price on 6th day)

In [16]:
# Assume we use 5 days price data to predict closing price of the 6th day
num_day_to_predict = 5

In [17]:
def produce_result_target_price(X,num_day = 5,result_col_name = "result_price"):
    y = pd.DataFrame(np.nan, index=X.index, columns=[result_col_name])
    for i in range(5,len(X)):
        y.loc[X.index[i-1],result_col_name] = X.loc[X.index[i],"open"]
    
    return y

In [18]:
train_y = produce_result_target_price(train_X,num_day_to_predict)
valid_y = produce_result_target_price(valid_X,num_day_to_predict)
test_y = produce_result_target_price(test_X,num_day_to_predict)

In [19]:
print(len(train_y))
print(len(valid_y))
print(len(test_y))

1218
292
505


In [20]:
pred_y = produce_result_target_price(stock,num_day_to_predict)

## Transform the 5-day data into one vector

In [21]:
def transform_x_data_to_one_vector(X,num_day):
    col_name = []
    for i in range(num_day):
        for j in X.columns:
            col_name.append(j+"-"+str(i))
    new_X = pd.DataFrame(np.nan, index=X.index, columns=col_name)
    
    for i in range(num_day-1,len(X)):
        for col in col_name:
            split_list = col.split("-")
            new_X.loc[X.index[i],col] = X.loc[X.index[i-int(split_list[1])],split_list[0]]
    
    return new_X

In [22]:
train_X_5 = transform_x_data_to_one_vector(train_X,num_day_to_predict)
valid_X_5 = transform_x_data_to_one_vector(valid_X,num_day_to_predict)
test_X_5 = transform_x_data_to_one_vector(test_X,num_day_to_predict)

In [23]:
pred_X_5 = transform_x_data_to_one_vector(stock,num_day_to_predict)

In [24]:
print(len(train_X_5))
print(len(valid_X_5))
print(len(test_X_5))
print(len(pred_X_5))

1218
292
505
2517


## Drop out rows with NaN

In [25]:
def drop_nan_row_y(y,num_day = 5):
    drop_list = [y.index[i] for i in range(num_day-1)]
    drop_list.append(y.index[-1])
    return y.drop(drop_list)

def drop_nan_row_X_5(X,num_day = 5):
    drop_list = [X.index[i] for i in range(num_day-1)]
    drop_list.append(X.index[-1])
    return X.drop(drop_list)

In [26]:
new_train_X_5 = drop_nan_row_X_5(train_X_5,num_day_to_predict)
new_train_y = drop_nan_row_y(train_y,num_day_to_predict)

new_valid_X_5  = drop_nan_row_X_5(valid_X_5,num_day_to_predict)
new_valid_y = drop_nan_row_y(valid_y,num_day_to_predict)

new_test_X_5  = drop_nan_row_X_5(test_X_5,num_day_to_predict)
new_test_y = drop_nan_row_y(test_y,num_day_to_predict)

In [27]:
new_pred_X_5  = drop_nan_row_X_5(pred_X_5,num_day_to_predict)
new_pred_y = drop_nan_row_y(pred_y,num_day_to_predict)

In [28]:
print(len(new_train_X_5))
print(len(new_valid_X_5))
print(len(new_test_X_5))
print(len(new_pred_X_5))

1213
287
500
2512


In [29]:
print(len(new_train_y))
print(len(new_valid_y))
print(len(new_test_y))
print(len(new_pred_y))

1213
287
500
2512


In [30]:
# Keep a copy for reverting prediction result
old_train_X_5 = new_train_X_5
old_train_y = new_train_y
old_valid_X_5 = new_valid_X_5
old_valid_y = new_valid_y
old_test_X_5 = new_test_X_5
old_test_y = new_test_y

In [31]:
old_pred_X_5 = new_pred_X_5
old_pred_y = new_pred_y

## Normalize data row by row

In [32]:
def normalize_data_by_row(X,y):
    norm_X = X.sub(X.mean(axis=0), axis=1).div(X.std(axis=0), axis=1)
    
    col_list =[] 
    for col in X.columns:
        if "open" in col:
            col_list.append(col)
    
    temp_y = y.loc[:,"result_price"]
    mean_5_day = np.mean(X.loc[:,col_list],axis=1)
    mean_5_day.index = temp_y.index
    std_5_day = np.std(X.loc[:,col_list],axis=1)
    std_5_day.index = temp_y.index
    

    norm_y = pd.DataFrame((temp_y-mean_5_day)/std_5_day,
                          columns=["result_price"])
                         
    return (norm_X,norm_y)

In [33]:
new_train_X_5,new_train_y = normalize_data_by_row(new_train_X_5,new_train_y)
new_valid_X_5,new_valid_y = normalize_data_by_row(new_valid_X_5,new_valid_y)
new_test_X_5,new_test_y = normalize_data_by_row(new_test_X_5,new_test_y)

In [34]:
new_pred_X_5,new_pred_y = normalize_data_by_row(new_pred_X_5,new_pred_y)

In [35]:
new_train_X_5 = drop_nan_row_X_5(new_train_X_5,num_day_to_predict)
new_valid_X_5 = drop_nan_row_X_5(new_valid_X_5,num_day_to_predict)
new_test_X_5 = drop_nan_row_X_5(new_test_X_5,num_day_to_predict)

In [36]:
new_train_y = drop_nan_row_y(new_train_y,num_day_to_predict)
new_valid_y = drop_nan_row_y(new_valid_y,num_day_to_predict)
new_test_y = drop_nan_row_y(new_test_y,num_day_to_predict)

In [37]:
new_train_X_5

,open-0,high-0,low-0,close-0,volume-0,boll-0,boll_ub-0,boll_lb-0,macd-0,macdh-0,...,volume-4,boll-4,boll_ub-4,boll_lb-4,macd-4,macdh-4,macds-4,rsi_11-4,rsi_14-4,rsi_21-4
20130114,0.801829,0.760783,0.804657,0.785525,0.061790,1.279803,1.427751,1.082076,-1.406745,-1.580071,...,-0.466321,1.174692,1.412356,0.891349,-1.474971,-0.776921,-1.319412,-1.322751,-1.378452,-1.373634
20130115,0.715955,0.727228,0.725498,0.780621,0.061379,1.304304,1.408701,1.149016,-1.298581,-1.627599,...,-0.381260,1.201114,1.421966,0.933513,-1.481067,-0.993095,-1.254608,-1.526228,-1.545811,-1.492648
20130116,0.756030,0.755054,0.781807,0.784707,-0.497657,1.329183,1.379580,1.226733,-1.138552,-1.519737,...,-0.198093,1.225358,1.417582,0.985378,-1.442880,-1.119914,-1.171975,-1.444668,-1.471530,-1.429014
20130117,0.859896,0.829532,0.890343,0.846824,-0.067296,1.352552,1.337385,1.314522,-0.921992,-1.202592,...,-0.072015,1.253790,1.420248,1.038409,-1.399309,-1.261002,-1.078883,-1.187923,-1.241027,-1.235013
20130118,0.871346,0.856540,0.921354,0.895047,0.240665,1.376592,1.294401,1.404411,-0.683810,-0.736428,...,0.063148,1.272926,1.418303,1.077832,-1.399416,-1.578803,-0.974154,-1.882927,-1.811044,-1.639197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20171023,0.441978,0.406401,0.342764,0.276328,0.453053,-0.186880,-0.234448,-0.132133,0.209160,1.457410,...,0.625585,-0.092756,0.049926,-0.231519,0.271695,0.170333,0.234060,-1.314137,-1.281820,-1.240553
20171024,0.288224,0.218979,0.053878,0.031946,1.415975,-0.212473,-0.344583,-0.072467,-0.046142,0.997030,...,10.375008,-0.079691,0.023243,-0.179297,0.769678,1.823307,0.220739,1.140162,1.083683,0.836491
20171025,0.043689,-0.001180,-0.085669,-0.123346,0.910779,-0.220528,-0.377332,-0.055597,-0.172709,0.836743,...,2.114572,-0.116833,-0.017453,-0.211431,0.693733,2.031001,0.071088,1.676509,1.547971,1.178749
20171026,-0.068355,-0.084660,-0.053026,-0.117625,-0.141216,-0.221283,-0.379406,-0.055008,-0.205863,0.937651,...,1.293473,-0.151671,-0.102481,-0.194808,0.513298,1.952254,-0.095693,2.190497,1.985631,1.492206


## Postprocessing after prediction

In [38]:
def convert_prediction(X,y,pred):
    col_list =[] 
    for col in X.columns:
        if "open" in col:
            col_list.append(col)
        
    mean_5_day = np.mean(X.loc[:,col_list],axis=1)
    mean_5_day.index = y.index
    std_5_day = np.std(X.loc[:,col_list],axis=1)
    std_5_day.index = y.index
    a = pd.DataFrame(index = y.index,columns=['mean','std','pred'])
    a.loc[:,"mean"] = mean_5_day
    a.loc[:,"std"] = std_5_day
    a.loc[:,"pred"] = pred
    a.loc[:,"converted"] = pred*std_5_day+mean_5_day
    return a["converted"]

In [39]:
def add_5_sma(stock_):
    stock_2 = stock_
    stock_2["5_sma"] = stock_2['open'].rolling(window=5,min_periods=1).mean()
    return(stock_2)

In [40]:
def convert_decision(stock_,pred):
    h = np.array(pred)
    action = []
    status = "N"
    five_sma = stock_['open'].rolling(window=5,min_periods=1).mean()

    ind_sma = five_sma.index
    ind_pred = pred.index
    union_ind = list(ind_sma.intersection(ind_pred))

    five_sma = five_sma.filter(items = union_ind)
    pred = pred.filter(items = union_ind)

    for i in range(len(union_ind)):
      
        if pred.iloc[i] >= five_sma.iloc[i]:
            if status == "N":
                action.append("Buy")
                status = "Buy"
            else:
                action.append("Hold")
        else:
            if status == "Buy":
                action.append("Sell")
                status = "N"
            else:
                action.append("Hold")

    result = (pd.DataFrame(action,
                         columns=["Action"], index = pred.index))
                
    return result

In [41]:
def standard_date_format(date_in_num):
    date = str(date_in_num)
    new_date = date[0:4] + "-" + date[4:6] + "-" + date[6:8] 
    return new_date

# Catboost Training

In [ ]:
pip install -q catboost

In [ ]:
from catboost import CatBoostRegressor

model_cat = CatBoostRegressor(random_state=1002,logging_level='Silent')

model_cat.fit(new_train_X_5, new_train_y, plot=True)
pred_cat = model_cat.predict(new_test_X_5, verbose=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
#Performance evaluation
print('CATBoost PERFORMANCE')
print('r2 score: '+str(r2_score(new_test_y, pred_cat)))
print('RMSE : '+str(np.sqrt(mean_squared_error(new_test_y, pred_cat))))
print("Mean Absolute Error : " + str(mean_absolute_error(new_test_y,pred_cat)))

CATBoost PERFORMANCE
r2 score: 0.5254815306534676
RMSE : 1.5517341798515667
Mean Absolute Error : 1.0839976510013503


In [ ]:
import pickle

In [ ]:
with open(model_folder+"Cat.pickle", "wb") as f:
    pickle.dump(model_cat, f)

In [ ]:
cat_final_pred = model_cat.predict(new_pred_X_5)

In [ ]:
cat_final_pred_converted = convert_prediction(old_pred_X_5,old_pred_y,cat_final_pred)
cat_final_pred_converted

20120109    124.970845
20120110    125.863163
20120111    125.842663
20120112    125.722518
20120113    124.862529
               ...    
20211223    131.639912
20211227    133.587578
20211228    133.281116
20211229    133.992156
20211230    135.724491
Name: converted, Length: 2512, dtype: float64

In [ ]:
action_cat = convert_decision(stock,cat_final_pred_converted)
action_cat['pred_price'] = cat_final_pred_converted.tolist()

In [ ]:
code = "AAPL"
file_name = f"CAT_{code}"
action_cat.to_csv(f"{file_name}.csv")

# Load Model

In [42]:
with open(model_folder+"Cat.pickle", "rb") as f:
    cat_model = pickle.load(f)

# Backtesting for Cat

In [44]:
import os

file_list = list(filter(lambda name: "CAT_" in name ,os.listdir(record_folder)))
if len(file_list) == 0:
  print("No such files...")
else:
  for f in file_list:
    print(file_list)
    os.remove(record_folder+f)
  print("All old files deleted...")

All old files deleted...


In [45]:
#### CAT Backtest
backtest_summary = pd.DataFrame(columns=backtest_col)
    
for sector, stocks in stock_list.items():
    for stock in stocks:
        print("Runninng stock: "+ stock)
        stock_data_1 = web.DataReader(stock, "stooq",start=start_date, end=end_date)
        stock_data_1.columns = ["open","high","low","close","volume"]
        x_1 = StockDataFrame(stock_data_1)
        data_1 = x_1[['open','high','low','close','volume',
                      'boll', 'boll_ub', 'boll_lb',
                      'macd', 'macdh', 'macds',
                      'rsi_11', 'rsi_14', 'rsi_21']]
        data_1.index = [int(str(ind)[0:4]+str(ind)[5:7]+str(ind)[8:10]) for ind in data_1.index]
        data_1 = data_1.sort_index()
        print(stock + " Data loaded...")
        
        for k,v in date_range_list.items():
         
            print("Getting Data... - " + stock + " from " + str(k))
            if (data_1.index[0]>v):
                print("No data for this testing period... - " + stock + " from " + str(k))
                continue
            test_X = custom_split(data_1,start = k,end = v)
            if len(test_X) <= num_day_to_predict:
                print("No enoguh data for testing... - " + stock + " from " + str(k))
                continue
                
            print("Transforming Data... - " + stock + " from " + str(k))
            test_y = produce_result_target_price(test_X,num_day_to_predict)
            test_X_5 = transform_x_data_to_one_vector(test_X,num_day_to_predict)

            new_test_X_5  = drop_nan_row_X_5(test_X_5,num_day_to_predict)

            new_test_y = drop_nan_row_y(test_y,num_day_to_predict)
            old_test_X_5 = new_test_X_5.copy()

            old_test_y = new_test_y.copy()
            new_test_X_5,new_test_y = normalize_data_by_row(new_test_X_5,new_test_y)

            print("Predicting... - " + stock + " from " + str(k))
            cat_prediction = cat_model.predict(new_test_X_5)
            
            print("Converting... - " + stock + " from " + str(k))
            pred_price = convert_prediction(old_test_X_5,old_test_y,cat_prediction)
            pred_action = convert_decision(data_1,pred_price)
            final_test_y = old_test_y.rename(columns={"result_price":"Open"})
            
            
            print("Backtesting... - " + stock + " from " + str(k))
            back1 = backtest(final_test_y,pred_action,10000,0.0005,period_year)
            back1.clear_trade_record(3)
            
            print("Start Iteration... - " + stock + " from " + str(k))
            back1.start_test()
            
            print("======= from "+ str(k) + " to " + str(v)+" =========")
            back1.print_do_nothing()
            back1.print_profit()
            back1.print_CAGR_performance()
            back1.export_trade_record("CAT_"+sector+"-"+stock,str(k)+"-to-"+str(v)+"("+str(round(back1.get_diff_in_CAGR()*100,2))+"%)")
            backtest_summary = backtest_summary.append({"Sector":sector,
                                                        "Stock":stock,
                                                        "Range_Start":str(k),
                                                        "Range_End":str(v),
                                                        do_nth_name+"_Profit":round(back1.get_do_nothing(),2),
                                                        model_name+"_Profit":round(back1.get_profit(),2),
                                                        do_nth_name+"_CAGR(in %)":round(back1.get_do_nothing_CAGR()*100,2),
                                                        model_name+"_CAGR(in %)":round(back1.get_model_CAGR()*100,2),
                                                        "CAGR_Performance(in %)":round(back1.get_diff_in_CAGR()*100,2),
                                                        do_nth_name+"_MDD(in %)":round(back1.get_do_nothing_MDD()*100,2),
                                                        model_name+"_MDD(in %)":round(back1.get_model_MDD()*100,2),
                                                        "MDD_Performance(in %)":round(back1.get_diff_in_MDD()*100,2)}, ignore_index=True)
            print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

print("Backtesting Completed...")
backtest_summary.to_csv(backtest_summary_folder+"backtest_summary_CAT.csv")
print("Backtesting Summary for CAT Exported...")

Streaming output truncated to the last 5000 lines.
Sold at 20150223 with price = 31.24 	Pos: 333
Bought at 20150224 with price = 31.58 	Pos: 330
Sold at 20150303 with price = 33.02 	Pos: 330
Bought at 20150309 with price = 32.89 	Pos: 331
Sold at 20150310 with price = 32.33 	Pos: 331
Bought at 20150313 with price = 32.23 	Pos: 331
Sold at 20150323 with price = 33.18 	Pos: 331
Bought at 20150402 with price = 32.81 	Pos: 335
Sold at 20150408 with price = 32.77 	Pos: 335
Bought at 20150415 with price = 31.93 	Pos: 343
Sold at 20150427 with price = 34.33 	Pos: 343
Bought at 20150511 with price = 33.72 	Pos: 349
Sold at 20150603 with price = 40.45 	Pos: 349
Bought at 20150604 with price = 39.31 	Pos: 359
Sold at 20150609 with price = 38.84 	Pos: 359
Bought at 20150612 with price = 38.59 	Pos: 361
Sold at 20150619 with price = 39.37 	Pos: 361
Bought at 20150710 with price = 39.13 	Pos: 362
Sold at 20150714 with price = 39.5 	Pos: 362
Bought at 20150728 with price = 37.0 	Pos: 387
Sold at 201